# Prepare Šolar and Lektor corpus error data


In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import pandas as pd

from utils.logging import get_logger
from utils.corpus_enum import Corpus
from utils.error_enum import ErrorType
from solar_prepare_data import SOLAR_DIRECTORY
from lektor_prepare_data import LEKTOR_DIRECTORY


In [ ]:
# Get logger
prepare_data_error_logger = get_logger("Prepare Corpus Error Data")


In [ ]:
# Constants
SOLAR_FILE_SINGLE_ERROR = (
    "../../data/solar/solar_data_single_error_raw.csv"  # solar data with single error
)
SOLAR_FILE_MULTIPLE_ERROR = "../../data/solar/solar_data_multiple_error_raw.csv"  # solar data with multiple error
LEKTOR_FILE_SINGLE_ERROR = "../../data/lektor/lektor_data_single_error_raw.csv"  # lektor data with single error
LEKTOR_FILE_MULTIPLE_ERROR = "../../data/lektor/lektor_data_multiple_error_raw.csv"  # solar data with multiple error
SOLAR_ERROR_TYPES = {
    # 1. NAPAKA ČRKOVANJA:
    "Č/VOK": "NAPAKA ČRKOVANJA: VOKALI",
    "Č/KONZ": "NAPAKA ČRKOVANJA: KONZONANTI",
    "Č/W": "NAPAKA ČRKOVANJA: USTIČNO-USTIČNI W",
    "Č/SKLOP": "NAPAKA ČRKOVANJA: ČRKOVNI SKLOPI",
    "Č/PRED": "NAPAKA ČRKOVANJA: VARIANTNI PREDLOGI",
    "Č/PREDL": "NAPAKA ČRKOVANJA: VARIANTNI PREDLOGI",  # this one is required due to inaccuracy in solar corpus
    # 2. NAPAKA OBLIKE:
    "O/KAT": "NAPAKA OBLIKE: KATEGORIALNI POPRAVKI",
    "O/PAR": "NAPAKA OBLIKE: PARADIGMATSKI POPRAVKI",
    "O/DOD": "NAPAKA OBLIKE: DODATNE OZNAKE",
    # 3. NAPAKA BESEDIŠČA:
    "B/SAM": "NAPAKA BESEDIŠČA: SAMOSTALNIK",
    "B/GLAG": "NAPAKA BESEDIŠČA: GLAGOL",
    "B/ZAIM": "NAPAKA BESEDIŠČA: ZAIMEK",
    "B/PRED": "NAPAKA BESEDIŠČA: PREDLOG",
    "B/VEZ": "NAPAKA BESEDIŠČA: VEZNIK",
    "B/PRID": "NAPAKA BESEDIŠČA: PRIDEVNIK",
    "B/PRISL": "NAPAKA BESEDIŠČA: PRISLOV",
    "B/OST": "NAPAKA BESEDIŠČA: OSTALE BESEDNE VRSTE",
    "B/MEN": "NAPAKA BESEDIŠČA: MENJAVA PREK MEJA BESEDNE VRSTE",
    "B/DOD": "NAPAKA BESEDIŠČA: DODATNE OZNAKE",
    # 4. NAPAKA SKLADNJE:
    "S/BR": "NAPAKA SKLADNJE: BESEDNI RED",
    "S/IZPUST": "NAPAKA SKLADNJE: IZPUŠČENI JEZIKOVNI ELEMENTI",
    "S/ODVEČ": "NAPAKA SKLADNJE: ODVEČNI JEZIKOVNI ELEMENTI",
    "S/STR": "NAPAKA SKLADNJE: STRUKTURA",
    "S/DOD": "NAPAKA SKLADNJE: DODATNE OZNAKE",
    # 5. NAPAKA ZAPISA:
    "Z/MV": "NAPAKA ZAPISA: MALA/VELIKA ZAČETNICA",
    "Z/SN": "NAPAKA ZAPISA: PISANJE SKUPAJ/NARAZEN",
    "Z/KR": "NAPAKA ZAPISA: KRAJŠAVA",
    "Z/ŠTEV": "NAPAKA ZAPISA: ŠTEVILO",
    "Z/LOČ": "NAPAKA ZAPISA: LOČILO",
    # 6. POVEZANA NAPAKA:
    "P/OBL": "POVEZANA NAPAKA: NAPAKA OBLIKE",
    "P/SKLA": "POVEZANA NAPAKA: NAPAKA SKLADNJE",
    "P/ZAP": "POVEZANA NAPAKA: NAPAKA ZAPISA",
    # 7. NEOPREDELJENA NAPAKA:
    "N/": "NEOPREDELJENA NAPAKA",
}
LEKTOR_ERROR_TYPES = {
    # 1. SLOGOVNA NAPAKA:
    "S-Dodajanje": "SLOGOVNA NAPAKA",
    "S-Drugo": "SLOGOVNA NAPAKA",
    "S-Dvojnica": "SLOGOVNA NAPAKA",
    "S-DvojnicaVarianta": "SLOGOVNA NAPAKA",
    "S-Izbris": "SLOGOVNA NAPAKA",
    "S-Kolokacija": "SLOGOVNA NAPAKA",
    "S-Koreferenca": "SLOGOVNA NAPAKA",
    "S-Pretvorba": "SLOGOVNA NAPAKA",
    "S-Prevzemanje": "SLOGOVNA NAPAKA",
    "S-Tujka": "SLOGOVNA NAPAKA",
    "S-Vezljivost": "SLOGOVNA NAPAKA",
    # 2. NAPAKA OBLIKE:
    "O-DomacaKrajevna": "NAPAKA OBLIKE",
    "O-DomacaOsebna": "NAPAKA OBLIKE",
    "O-Funkcijska": "NAPAKA OBLIKE",
    "O-Glagoli": "NAPAKA OBLIKE",
    "O-Pridevniki": "NAPAKA OBLIKE",
    "O-StvarnaObcna": "NAPAKA OBLIKE",
    "O-Stevniki": "NAPAKA OBLIKE",
    "O-TujaKrajevna": "NAPAKA OBLIKE",
    "O-TujaOsebna": "NAPAKA OBLIKE",
    "O-Zaimki": "NAPAKA OBLIKE",
    # 3. PRAVOPISNA NAPAKA:
    "pravopis": "PRAVOPISNA NAPAKA",
    "P-Izraz": "PRAVOPISNA NAPAKA",
    "P-Krajsava": "PRAVOPISNA NAPAKA",
    "P-LociloStava": "PRAVOPISNA NAPAKA",
    "P-LociloZamenjava": "PRAVOPISNA NAPAKA",
    "P-SkupajNarazen": "PRAVOPISNA NAPAKA",
    "P-Tipkarska": "PRAVOPISNA NAPAKA",
    "P-Tvorjenka": "PRAVOPISNA NAPAKA",
    "P-ZacetnicaBitja": "PRAVOPISNA NAPAKA",
    "P-ZacetnicaKrajevno": "PRAVOPISNA NAPAKA",
    "P-ZacetnicaPridevnik": "PRAVOPISNA NAPAKA",
    "P-ZacetnicaStavcna": "PRAVOPISNA NAPAKA",
    "P-ZacetnicaStvarno": "PRAVOPISNA NAPAKA",
    "P-ZacetnicaStvarnaObcna": "PRAVOPISNA NAPAKA",
    "P-Zapis": "PRAVOPISNA NAPAKA",
    "predlog": "PRAVOPISNA NAPAKA",
    "veznik": "PRAVOPISNA NAPAKA",
    # 4. NAPAKA SKLADNJE:
    "skladnja": "NAPAKA SKLADNJE",
    "Sk-BesedniRed": "NAPAKA SKLADNJE",
    "Sk-Drugo": "NAPAKA SKLADNJE",
    "Sk-Predlog": "NAPAKA SKLADNJE",
    "Sk-PretvorbaTvorno": "NAPAKA SKLADNJE",
    "Sk-PretvorbaNeosebno": "NAPAKA SKLADNJE",
    "Sk-Razvezava": "NAPAKA SKLADNJE",
    "Sk-SkladenjskaPretvorba": "NAPAKA SKLADNJE",
    "Sk-Ujemanje": "NAPAKA SKLADNJE",
    "Sk-Vezava": "NAPAKA SKLADNJE",
    "Sk-Veznik": "NAPAKA SKLADNJE",
    "Sk-Zdruzitev": "NAPAKA SKLADNJE",
    # 5. NAPAKA PRAGMATIKE:
    "Pr-Faktografija": "NAPAKA PRAGMATIKE",
    "Pr-Komentar": "NAPAKA PRAGMATIKE",
    "Pr-Pomen": "NAPAKA PRAGMATIKE",
    "Pr-Pomota": "NAPAKA PRAGMATIKE",
    "Pr-Povezana": "NAPAKA PRAGMATIKE",
    "Pr-Prevajalska": "NAPAKA PRAGMATIKE",
}


In [ ]:
def read_error_data(corpus=Corpus.SOLAR, error=ErrorType.SINGLE):
    """
    Reads the solar or lektor error data and returns a data frame.

    @param corpus: corpus type (solar or lektor)
    @param error: error file type (single error or multiple errors)
    @return: data frame with the corpus error data
    """
    data_path = SOLAR_FILE_SINGLE_ERROR
    if corpus == Corpus.LEKTOR:
        if error == ErrorType.MULTIPLE:
            data_path = LEKTOR_FILE_MULTIPLE_ERROR
        else:
            data_path = LEKTOR_FILE_SINGLE_ERROR
    else:
        if error == ErrorType.MULTIPLE:
            data_path = SOLAR_FILE_MULTIPLE_ERROR

    # Read the solar or lektor error data from the file
    data = pd.read_csv(data_path, keep_default_na=False)
    prepare_data_error_logger.info("Corpus error data read")

    return data


In [ ]:
def split_error_data(data):
    """
    Splits the combined error types into separate error types and returns a data frame.

    @param data: data frame with the corpus error data
    @return: data frame with the corpus error data split into separate error types
    """
    temp_data = []

    # Loop through the error types and add create new cases with combined error types
    for temp_sentence_id, temp_sentence, temp_errors in data.itertuples(index=False):
        for temp_error in temp_errors.split("|"):
            temp_data.append([temp_sentence_id, temp_sentence, temp_error])

    # Create a new data frame with the error types
    data = pd.DataFrame(data=temp_data, columns=data.columns)
    return data


In [ ]:
def rename_solar_error_type(error=ErrorType.SINGLE):
    """
    Reassigns the solar error types to the ones in SOLAR_ERROR_TYPES.

    @param error: error file type (single error or multiple errors)
    @return: data frame with the corpus error data with the new error types
    """
    # Read corpus data
    data = read_error_data(Corpus.SOLAR, error)

    errors = []
    for sentence in data.itertuples():
        # [Index, id, sentence, error]
        error = []

        for error_codes in sentence[3].split(";"):
            temp_error = []

            # Replace all the error types with their corresponding error type
            for error_code in error_codes.split("|"):
                if error_code == "":
                    continue

                error_code = "/".join(error_code.split("/")[:-1])
                temp_error.append(SOLAR_ERROR_TYPES[error_code])

            error.append("|".join(set(temp_error)))

        # Remove duplicate error types and join them
        errors.append(";".join(set(error)))

    data["error"] = errors

    if error == ErrorType.SINGLE:
        # Split the error types into separate error types
        data = split_error_data(data)
        prepare_data_error_logger.info("Solar error types renamed")

    return data


In [ ]:
def rename_lektor_error_type(error=ErrorType.SINGLE):
    """
    Reassigns the lektor error types to the ones in LEKTOR_ERROR_TYPES.

    @param error: error file type (single error or multiple errors)
    @return: data frame with the corpus error data with the new error types
    """
    data = read_error_data(Corpus.LEKTOR, error)

    errors = []
    for sentence in data.itertuples():
        # [Index, id, sentence, error]
        error = []

        # Replace all the error types with their corresponding error type
        error_codes = sentence[3].split(";")
        for error_code in error_codes:
            if error_code == "":
                continue

            error_code = "-".join(error_code.split("-")[:2])
            error.append(LEKTOR_ERROR_TYPES[error_code])

        # Remove duplicate error types and join them
        errors.append("|".join(set(error)))

    data["error"] = errors

    if error == ErrorType.SINGLE:
        # Split the error types into separate error types
        data = split_error_data(data)
        prepare_data_error_logger.info("Lektor error types renamed")

    return data


In [ ]:
def save_error_data():
    """
    Saves the solar and lektor error data to the csv file.
    """
    # Get the data
    data_solar_single = rename_solar_error_type(ErrorType.SINGLE)
    data_solar_multiple = rename_solar_error_type(ErrorType.MULTIPLE)
    data_lektor_single = rename_lektor_error_type(ErrorType.SINGLE)
    data_lektor_multiple = rename_lektor_error_type(ErrorType.MULTIPLE)

    data_solar_single.to_csv(
        SOLAR_DIRECTORY + "solar_data_single_error_all.csv", index=False
    )
    data_solar_multiple.to_csv(
        SOLAR_DIRECTORY + "solar_data_multiple_error_all.csv", index=False
    )
    prepare_data_error_logger.info("Solar data saved to csv file")

    data_lektor_single.to_csv(
        LEKTOR_DIRECTORY + "lektor_data_single_error_all.csv", index=False
    )
    data_lektor_multiple.to_csv(
        LEKTOR_DIRECTORY + "lektor_data_multiple_error_all.csv", index=False
    )
    prepare_data_error_logger.info("Lektor data saved to csv file")
    return
